In [166]:
import pandas as pd

data = pd.read_csv("../0-Data/train_titanic.csv")
pd.set_option("display.max_columns", len(data.columns))
print(len(data.columns))
data.head()

12


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [167]:
data.dtypes

PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object

In [168]:
drop = ["Cabin", "Name", "Ticket", "PassengerId"]
data = data.drop(drop, axis=1)
data["Embarked"] = data["Embarked"].fillna(data["Embarked"].mode())

toEncode = [column for column in data.columns if data[column].dtype == object]
for i in toEncode:
    encoded = pd.get_dummies(data[i], dtype=float)
    data = pd.concat([data, encoded], axis=1)
    data.drop(i, axis=1, inplace=True)
pd.set_option("display.max_columns", len(data.columns))
print(len(data.columns))
data.dtypes

11


Survived      int64
Pclass        int64
Age         float64
SibSp         int64
Parch         int64
Fare        float64
female      float64
male        float64
C           float64
Q           float64
S           float64
dtype: object

In [169]:
data.isna().sum() / len(data) * 100

Survived     0.00000
Pclass       0.00000
Age         19.86532
SibSp        0.00000
Parch        0.00000
Fare         0.00000
female       0.00000
male         0.00000
C            0.00000
Q            0.00000
S            0.00000
dtype: float64

In [170]:
a_train = data[data["Age"].notnull()]
b_train = a_train["Age"]
a_train = a_train.drop("Age", axis=1)
a_fill = data[data["Age"].isnull()]
b_fill = a_fill["Age"]
a_fill = a_fill.drop("Age", axis=1)

from sklearn.tree import DecisionTreeRegressor

model = DecisionTreeRegressor()
model.fit(a_train, b_train)

y_fill = model.predict(a_fill)

data.loc[data["Age"].isnull(), "Age"] = y_fill
data.isna().sum() / len(data) * 100

Survived    0.0
Pclass      0.0
Age         0.0
SibSp       0.0
Parch       0.0
Fare        0.0
female      0.0
male        0.0
C           0.0
Q           0.0
S           0.0
dtype: float64

In [171]:
x = data.drop("Survived", axis=1)
y = data["Survived"]

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
import numpy as np

skfold = StratifiedKFold(n_splits=4, shuffle=True)

param = {
    "learning_rate": np.array([0.1, 0.075, 0.05]),
    "min_samples_split": np.array([2, 3]),
    "max_depth": np.array([2, 3]),
    "min_samples_leaf": np.array([1, 2]),
}
model = GradientBoostingClassifier(n_estimators=100)
grid_model = GridSearchCV(estimator=model, param_grid=param, cv=skfold, n_jobs=-1)
grid_model.fit(x, y)

print(f"Best learning rate:", grid_model.best_estimator_.learning_rate)
print(f"Best samples split:", grid_model.best_estimator_.min_samples_split)
print(f"Best depth: {grid_model.best_estimator_.max_depth}")
print(f"Best samples leaf: {grid_model.best_estimator_.min_samples_leaf}")
print(f"R2: {grid_model.best_score_}")

Best learning rate: 0.1
Best samples split: 3
Best depth: 3
Best samples leaf: 1
R2: 0.8417767543328082
